In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# -*- encoding:utf-8 -*-
"""
  This script provides an k-BERT exmaple for classification.
"""
import sys
import torch
import json
import random
import argparse
import collections
import torch.nn as nn
from uer.utils.vocab import Vocab
from uer.utils.constants import *
from uer.utils.tokenizer import * 
from uer.model_builder import build_model
from uer.utils.optimizers import  BertAdam
from uer.utils.config import load_hyperparam  
from uer.utils.seed import set_seed
from uer.model_saver import save_model
from multiprocessing import Process, Pool
import numpy as np
import os
import sys
import pandas as pd
import inspect
from tqdm import tqdm

In [3]:
# df_train_chinese = pd.read_csv('../../NLP_DATA/multilingual_data/200410_train_stratshuf_chinese.csv')
# df_test_chinese = pd.read_csv('../../NLP_DATA/multilingual_data/200410_test_stratshuf_chinese.csv')


In [4]:
# df_train_chinese.rename({'text':'text_a','labels':'label'}, axis=1, inplace=True)
# df_train_chinese[['text_a', 'label']].to_csv('huawei_train.csv', sep='\t',index=False )

In [5]:
# df_test_chinese.rename({'text':'text_a','labels':'label'}, axis=1, inplace=True)
# df_test_chinese[['text_a', 'label']].to_csv('huawei_test.csv', sep='\t',index=False )

In [6]:
class huawei():
    def __init__(self):
        self.pretrained_model_path='./models/google_model.bin'
        self.output_model_path="./models/classifier_model.bin"
        self.vocab_path="./models/google_vocab.txt"
        self.train_path='./huawei_train.csv'
        self.config_path = './models/google_config.json'
        self.vocab_path = './models/google_vocab.txt'
        self.dev_path = './huawei_test.csv'
        self.test_path ='./huawei_test.csv'
        self.epochs_num = 35
        self.batch_size =  30
        self.kg_name = 'CnDbpedia'#'short_CnDbpedia'
        self.output_model_path= './outputs/kbert_huawei_CnDbpedia.bin'
        self.pooling='last'#'first'

        self.seq_length=140
        self.encoder="bert"
        self.bidirectional=True #"store_true"

        # Subword options.
        self.subword_type= 'none'#"char"#'none'
        self.sub_vocab_path = "models/sub_vocab.txt"
        self.subencoder="avg" #choices=["avg", "lstm", "gru", "cnn"] #sota avg
        self.sub_layers_num=2

        # Tokenizer options.
        self.tokenizer="bert"
            

        # Optimizer options.
        self.learning_rate =2e-5
                   
        self.warmup=0.1
                

        # Training options.
        self.dropout=default=0.5

                   
        self.report_steps=100
        self.seed=7
        # Evaluation options.
        self.mean_reciprocal_rank=False#'store_true'
        # kg
        self.workers_num=1#, help="number of process for loading dataset")
        self.no_vm= False #'store_true'#, help="Disable the visible_matrix")

# args = parser.parse_args()

# Load the hyperparameters from the config file.


In [7]:
# args = argsp()
args=huawei()

In [8]:
args = load_hyperparam(args)

In [9]:
inspect.getmembers(args)

[('__class__', __main__.huawei),
 ('__delattr__',
  <method-wrapper '__delattr__' of huawei object at 0x7f55cc4b0588>),
 ('__dict__',
  {'pretrained_model_path': './models/google_model.bin',
   'output_model_path': './outputs/kbert_huawei_CnDbpedia.bin',
   'vocab_path': './models/google_vocab.txt',
   'train_path': './huawei_train.csv',
   'config_path': './models/google_config.json',
   'dev_path': './huawei_test.csv',
   'test_path': './huawei_test.csv',
   'epochs_num': 35,
   'batch_size': 30,
   'kg_name': 'CnDbpedia',
   'pooling': 'last',
   'seq_length': 140,
   'encoder': 'bert',
   'bidirectional': True,
   'subword_type': 'none',
   'sub_vocab_path': 'models/sub_vocab.txt',
   'subencoder': 'avg',
   'sub_layers_num': 2,
   'tokenizer': 'bert',
   'learning_rate': 2e-05,
   'warmup': 0.1,
   'dropout': 0.1,
   'report_steps': 100,
   'seed': 7,
   'mean_reciprocal_rank': False,
   'workers_num': 1,
   'no_vm': False,
   'emb_size': 768,
   'hidden_size': 768,
   'kernel_siz

In [10]:
# FILE_DIR_PATH = os.path.dirname(os.path.abspath(__file__))

# KGS = {
#     'HowNet': os.path.join(FILE_DIR_PATH, 'kgs/HowNet.spo'),
#     'CnDbpedia': os.path.join(FILE_DIR_PATH, 'kgs/CnDbpedia.spo'),
#     'Medical': os.path.join(FILE_DIR_PATH, 'kgs/Medical.spo'),
# }

# MAX_ENTITIES = 2

# # Special token words.
# PAD_TOKEN = '[PAD]'
# UNK_TOKEN = '[UNK]'
# CLS_TOKEN = '[CLS]'
# SEP_TOKEN = '[SEP]'
# MASK_TOKEN = '[MASK]'
# ENT_TOKEN = '[ENT]'
# SUB_TOKEN = '[SUB]'
# PRE_TOKEN = '[PRE]'
# OBJ_TOKEN = '[OBJ]'

# NEVER_SPLIT_TAG = [
#     PAD_TOKEN, UNK_TOKEN, CLS_TOKEN, SEP_TOKEN, MASK_TOKEN,
#     ENT_TOKEN, SUB_TOKEN, PRE_TOKEN, OBJ_TOKEN
# ]


In [11]:
sys.path.insert(0, './brain')

In [12]:
# coding: utf-8
"""
KnowledgeGraph
"""
import os
import config
import pkuseg
import numpy as np

In [13]:
# from knowgraph import KnowledgeGraph
class KnowledgeGraph(object):
    """
    spo_files - list of Path of *.spo files, or default kg name. e.g., ['HowNet']
    """

    def __init__(self, spo_files, predicate=False):
        self.predicate = predicate
        self.spo_file_paths = [config.KGS.get(f, f) for f in spo_files]
        self.lookup_table = self._create_lookup_table()
        print('extracting keys')
        self.segment_vocab = list(self.lookup_table.keys()) + config.NEVER_SPLIT_TAG
        print('tokenize')
        self.tokenizer = pkuseg.pkuseg(model_name="default", postag=False, user_dict=self.segment_vocab) #just initialise
        print('end tokenize')
        self.special_tags = set(config.NEVER_SPLIT_TAG)
        self.entities = {}

    def _create_lookup_table(self):
        lookup_table = {}
        for spo_path in self.spo_file_paths:
            print("[KnowledgeGraph] Loading spo from {}".format(spo_path))
            with open(spo_path, 'r', encoding='utf-8') as f:
                for line in tqdm(f):
                    try:
                        subj, pred, obje = line.strip().split("\t")    
                    except:
                        print("[KnowledgeGraph] Bad spo:", line)
                    if self.predicate:
                        value = pred + obje
                    else:
                        value = obje
                    if subj in lookup_table.keys():
                        lookup_table[subj].add(value)
                    else:
                        lookup_table[subj] = set([value])
        return lookup_table

    def add_knowledge_with_vm(self, sent_batch, max_entities=config.MAX_ENTITIES, add_pad=True, max_length=128, debug = False):
        """
        input: sent_batch - list of sentences, e.g., ["abcd", "efgh"]
        entities is the set of detected entities in the train.
        return: know_sent_batch - list of sentences with entities embedding
                position_batch - list of position index of each character.
                visible_matrix_batch - list of visible matrixs
                seg_batch - list of segment tags
        """
        split_sent_batch = [self.tokenizer.cut(sent) for sent in sent_batch]
        know_sent_batch = []
        position_batch = []
        visible_matrix_batch = []
        seg_batch = []
        for split_sent in split_sent_batch:

            # create tree
            sent_tree = []
            pos_idx_tree = []
            abs_idx_tree = []
            pos_idx = -1
            abs_idx = -1
            abs_idx_src = []
            for token in split_sent:

                entities = list(self.lookup_table.get(token, []))[:max_entities]
                if debug: 
                    print(entities)  ######
                if token in self.entities.keys():
                    self.entities[token].extend(entities)
                else:
                    self.entities[token]=entities
                    
                sent_tree.append((token, entities))

                if token in self.special_tags:
                    token_pos_idx = [pos_idx+1]
                    token_abs_idx = [abs_idx+1]
                else:
                    token_pos_idx = [pos_idx+i for i in range(1, len(token)+1)]
                    token_abs_idx = [abs_idx+i for i in range(1, len(token)+1)]
                abs_idx = token_abs_idx[-1]

                entities_pos_idx = []
                entities_abs_idx = []
                for ent in entities:
                    ent_pos_idx = [token_pos_idx[-1] + i for i in range(1, len(ent)+1)]
                    entities_pos_idx.append(ent_pos_idx)
                    ent_abs_idx = [abs_idx + i for i in range(1, len(ent)+1)]
                    abs_idx = ent_abs_idx[-1]
                    entities_abs_idx.append(ent_abs_idx)

                pos_idx_tree.append((token_pos_idx, entities_pos_idx))
                pos_idx = token_pos_idx[-1]
                abs_idx_tree.append((token_abs_idx, entities_abs_idx))
                abs_idx_src += token_abs_idx

            # Get know_sent and pos
            know_sent = []
            pos = []
            seg = []
            for i in range(len(sent_tree)):
                word = sent_tree[i][0]
                if word in self.special_tags:
                    know_sent += [word]
                    seg += [0]
                else:
                    add_word = list(word)
                    know_sent += add_word 
                    seg += [0] * len(add_word)
                pos += pos_idx_tree[i][0]
                for j in range(len(sent_tree[i][1])):
                    add_word = list(sent_tree[i][1][j])
                    know_sent += add_word
                    seg += [1] * len(add_word)
                    pos += list(pos_idx_tree[i][1][j])

            token_num = len(know_sent)

            # Calculate visible matrix
            visible_matrix = np.zeros((token_num, token_num))
            for item in abs_idx_tree:
                src_ids = item[0]
                for id in src_ids:
                    visible_abs_idx = abs_idx_src + [idx for ent in item[1] for idx in ent]
                    visible_matrix[id, visible_abs_idx] = 1
                for ent in item[1]:
                    for id in ent:
                        visible_abs_idx = ent + src_ids
                        visible_matrix[id, visible_abs_idx] = 1

            src_length = len(know_sent)
            if len(know_sent) < max_length:
                pad_num = max_length - src_length
                know_sent += [config.PAD_TOKEN] * pad_num
                seg += [0] * pad_num
                pos += [max_length - 1] * pad_num
                visible_matrix = np.pad(visible_matrix, ((0, pad_num), (0, pad_num)), 'constant')  # pad 0
            else:
                know_sent = know_sent[:max_length]
                seg = seg[:max_length]
                pos = pos[:max_length]
                visible_matrix = visible_matrix[:max_length, :max_length]
            
            know_sent_batch.append(know_sent)
            position_batch.append(pos)
            visible_matrix_batch.append(visible_matrix)
            seg_batch.append(seg)
        
        return know_sent_batch, position_batch, visible_matrix_batch, seg_batch


In [14]:
class BertClassifier(nn.Module):
    """
    Custom BERT wrapper.
    """
    def __init__(self, args, model):
        super(BertClassifier, self).__init__()
        self.embedding = model.embedding
        self.encoder = model.encoder
        self.labels_num = args.labels_num
        self.pooling = args.pooling
        self.output_layer_1 = nn.Linear(args.hidden_size, args.hidden_size)
        self.output_layer_2 = nn.Linear(args.hidden_size, args.labels_num)
        self.softmax = nn.LogSoftmax(dim=-1)
        self.criterion = nn.NLLLoss()
        self.use_vm = False if args.no_vm else True
        print("[BertClassifier] use visible_matrix: {}".format(self.use_vm))

    def forward(self, src, label, mask, pos=None, vm=None):
        """
        Args:
            src: [batch_size x seq_length]
            label: [batch_size]
            mask: [batch_size x seq_length]
        """
        # Embedding.
        emb = self.embedding(src, mask, pos)
        # Encoder.
        if not self.use_vm:
            vm = None
        output = self.encoder(emb, mask, vm)
        # Target.
        if self.pooling == "mean":
            output = torch.mean(output, dim=1)
        elif self.pooling == "max":
            output = torch.max(output, dim=1)[0]
        elif self.pooling == "last":
            output = output[:, -1, :]
        else:
            output = output[:, 0, :]
        output = torch.tanh(self.output_layer_1(output))
        logits = self.output_layer_2(output)
        loss = self.criterion(self.softmax(logits.view(-1, self.labels_num)), label.view(-1))
        return loss, logits

In [15]:
set_seed(args.seed)

# Count the number of labels.
labels_set = set()
columns = {}
with open(args.train_path, mode="r", encoding="utf-8") as f:
    for line_id, line in enumerate(f):
        try:
            line = line.strip().split("\t")
            if line_id == 0:
                for i, column_name in enumerate(line):
                    columns[column_name] = i
                continue
            label = int(line[columns["label"]])
            labels_set.add(label)
        except:
            pass
args.labels_num = len(labels_set) 

# Load vocabulary.
vocab = Vocab()
vocab.load(args.vocab_path)
args.vocab = vocab

# Build bert model.
# A pseudo target is added.
args.target = "bert"
model = build_model(args)  ##USES UER FRAMEWORK

# Load or initialize parameters.
if args.pretrained_model_path is not None:
    # Initialize with pretrained model.
    model.load_state_dict(torch.load(args.pretrained_model_path), strict=False)  
else:
    # Initialize with normal distribution.
    for n, p in list(model.named_parameters()):
        if 'gamma' not in n and 'beta' not in n:
            p.data.normal_(0, 0.02)

# Build classification model.
model = BertClassifier(args, model)

# For simplicity, we use DataParallel wrapper to use multiple GPUs.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("{} GPUs are available. Let's use them.".format(torch.cuda.device_count()))
    model = nn.DataParallel(model)

model = model.to(device)



Vocabulary file line 344 has bad format token
Vocabulary Size:  21128
[BertClassifier] use visible_matrix: True


In [16]:
args.labels_num = len(labels_set) 

In [17]:
args.labels_num 

336

## Datset loader

In [18]:

def batch_loader(batch_size, input_ids, label_ids, mask_ids, pos_ids, vms):
    """
    Just takes inputs, labels, masks, positions and VM to create batch. Nothing more.
    """
    instances_num = input_ids.size()[0]
    for i in range(instances_num // batch_size):
        input_ids_batch = input_ids[i*batch_size: (i+1)*batch_size, :]
        label_ids_batch = label_ids[i*batch_size: (i+1)*batch_size]
        mask_ids_batch = mask_ids[i*batch_size: (i+1)*batch_size, :]
        pos_ids_batch = pos_ids[i*batch_size: (i+1)*batch_size, :]
        vms_batch = vms[i*batch_size: (i+1)*batch_size]
        yield input_ids_batch, label_ids_batch, mask_ids_batch, pos_ids_batch, vms_batch
    if instances_num > instances_num // batch_size * batch_size:
        input_ids_batch = input_ids[instances_num//batch_size*batch_size:, :]
        label_ids_batch = label_ids[instances_num//batch_size*batch_size:]
        mask_ids_batch = mask_ids[instances_num//batch_size*batch_size:, :]
        pos_ids_batch = pos_ids[instances_num//batch_size*batch_size:, :]
        vms_batch = vms[instances_num//batch_size*batch_size:]

        yield input_ids_batch, label_ids_batch, mask_ids_batch, pos_ids_batch, vms_batch

# Build knowledge graph.
if args.kg_name == 'none':
    spo_files = []
else:
    spo_files = [args.kg_name]

In [19]:
kg = KnowledgeGraph(spo_files=spo_files, predicate=True)

41920it [00:00, 419197.40it/s]

[KnowledgeGraph] Loading spo from /home/aiops/EDWARD/nlp_code/K-BERT/brain/kgs/CnDbpedia.spo


5168865it [00:08, 591507.65it/s]


extracting keys
tokenize
end tokenize


In [20]:
# inspect.getmembers(kg)

In [21]:
#SHOW THE KNOWLEDGE
# lim = {k:set(kg.entities[k]) for  k in kg.entities.keys() if len(set(kg.entities[k]))>0}

In [22]:
# lim

In [23]:
def add_knowledge_worker(params, debug=False):
    '''
    This determines if it is a sentence classification or similarity of 2 sentences type task.
    '''
    p_id, sentences, columns, kg, vocab, args = params

    sentences_num = len(sentences)
    dataset = []
    for line_id, line in enumerate(sentences):
        if line_id % 10000 == 0:
            print("Progress of process {}: {}/{}".format(p_id, line_id, sentences_num))
            sys.stdout.flush()
        line = line.strip().split('\t')
        try:
            if len(line) == 2:
                if debug: print(columns, line) ########
                label = int(line[columns["label"]])
                text = CLS_TOKEN + line[columns["text_a"]]
   
                tokens, pos, vm, _ = kg.add_knowledge_with_vm([text], add_pad=True, max_length=args.seq_length)
                tokens = tokens[0]
                pos = pos[0]
                vm = vm[0].astype("bool")
                if debug: print(tokens) #######
                token_ids = [vocab.get(t) for t in tokens]
                mask = [1 if t != PAD_TOKEN else 0 for t in tokens]

                dataset.append((token_ids, label, mask, pos, vm))
            
            elif len(line) == 3:
                label = int(line[columns["label"]])
                text = CLS_TOKEN + line[columns["text_a"]] + SEP_TOKEN + line[columns["text_b"]] + SEP_TOKEN

                tokens, pos, vm, _ = kg.add_knowledge_with_vm([text], add_pad=True, max_length=args.seq_length)
                tokens = tokens[0]
                pos = pos[0]
                vm = vm[0].astype("bool")

                token_ids = [vocab.get(t) for t in tokens]
                mask = []
                seg_tag = 1
                for t in tokens:
                    if t == PAD_TOKEN:
                        mask.append(0)
                    else:
                        mask.append(seg_tag)
                    if t == SEP_TOKEN:
                        seg_tag += 1

                dataset.append((token_ids, label, mask, pos, vm))
            
            elif len(line) == 4:  # for dbqa
                qid=int(line[columns["qid"]])
                label = int(line[columns["label"]])
                text_a, text_b = line[columns["text_a"]], line[columns["text_b"]]
                text = CLS_TOKEN + text_a + SEP_TOKEN + text_b + SEP_TOKEN

                tokens, pos, vm, _ = kg.add_knowledge_with_vm([text], add_pad=True, max_length=args.seq_length)
                tokens = tokens[0]
                pos = pos[0]
                vm = vm[0].astype("bool")

                token_ids = [vocab.get(t) for t in tokens]
                mask = []
                seg_tag = 1
                for t in tokens:
                    if t == PAD_TOKEN:
                        mask.append(0)
                    else:
                        mask.append(seg_tag)
                    if t == SEP_TOKEN:
                        seg_tag += 1
                
                dataset.append((token_ids, label, mask, pos, vm, qid))
            else:
                pass
            
        except Exception as e:
            print("Error line: ", line, e)
    return dataset

In [24]:

def read_dataset(path, workers_num=1):
    """
    runs add_knowledge_worker on dataset
    """
    print("Loading sentences from {}".format(path))
    sentences = []
    with open(path, mode='r', encoding="utf-8") as f:
        for line_id, line in enumerate(f):
            if line_id == 0:
                continue
            sentences.append(line)
    sentence_num = len(sentences)

    print("There are {} sentence in total. We use {} processes to inject knowledge into sentences.".format(sentence_num, workers_num))
    if workers_num > 1:
        params = []
        sentence_per_block = int(sentence_num / workers_num) + 1
        for i in range(workers_num):
            params.append((i, sentences[i*sentence_per_block: (i+1)*sentence_per_block], columns, kg, vocab, args))
        pool = Pool(workers_num)
        res = pool.map(add_knowledge_worker, params)
        pool.close()
        pool.join()
        dataset = [sample for block in res for sample in block]
    else:
        params = (0, sentences, columns, kg, vocab, args)
        dataset = add_knowledge_worker(params)

    return dataset

# Evaluation function.
def evaluate(args, is_test, metrics='Acc'):
    if is_test:
        dataset = read_dataset(args.test_path, workers_num=args.workers_num)
    else:
        dataset = read_dataset(args.dev_path, workers_num=args.workers_num)

    input_ids = torch.LongTensor([sample[0] for sample in dataset])
    label_ids = torch.LongTensor([sample[1] for sample in dataset])
    mask_ids = torch.LongTensor([sample[2] for sample in dataset])
    pos_ids = torch.LongTensor([example[3] for example in dataset])
    vms = [example[4] for example in dataset]

    batch_size = args.batch_size
    instances_num = input_ids.size()[0]
    if is_test:
        print("The number of evaluation instances: ", instances_num)

    correct = 0
    # Confusion matrix.
    confusion = torch.zeros(args.labels_num, args.labels_num, dtype=torch.long)

    model.eval()

    if not args.mean_reciprocal_rank:
        print('not MMR')
        for i, (input_ids_batch, label_ids_batch,  mask_ids_batch, pos_ids_batch, vms_batch) in enumerate(batch_loader(batch_size, input_ids, label_ids, mask_ids, pos_ids, vms)):

            # vms_batch = vms_batch.long()
            vms_batch = torch.LongTensor(vms_batch)

            input_ids_batch = input_ids_batch.to(device)
            label_ids_batch = label_ids_batch.to(device)
            mask_ids_batch = mask_ids_batch.to(device)
            pos_ids_batch = pos_ids_batch.to(device)
            vms_batch = vms_batch.to(device)

            with torch.no_grad():
                try:
                    loss, logits = model(input_ids_batch, label_ids_batch, mask_ids_batch, pos_ids_batch, vms_batch)
                except:
                    print(input_ids_batch)
                    print(input_ids_batch.size())
                    print(vms_batch)
                    print(vms_batch.size())

            logits = nn.Softmax(dim=1)(logits)
            pred = torch.argmax(logits, dim=1)
            gold = label_ids_batch
#             for j in range(pred.size()[0]):
#                 confusion[pred[j], gold[j]] += 1
            correct += torch.sum(pred == gold).item()

#         if is_test:
#             print("Confusion matrix:")
#             print(confusion)
#             print("Report precision, recall, and f1:")

#         for i in range(confusion.size()[0]):
#             p = confusion[i,i].item()/confusion[i,:].sum().item()
#             r = confusion[i,i].item()/confusion[:,i].sum().item()
#             f1 = 2*p*r / (p+r)
#             if i == 1:
#                 label_1_f1 = f1
#             print("Label {}: {:.3f}, {:.3f}, {:.3f}".format(i,p,r,f1))
        print("Acc. (Correct/Total): {:.4f} ({}/{}) ".format(correct/len(dataset), correct, len(dataset)))
        if metrics == 'Acc':
            return correct/len(dataset)
        elif metrics == 'f1':
            return label_1_f1
        else:
            return correct/len(dataset)
    else:
        print('mrr')
        for i, (input_ids_batch, label_ids_batch,  mask_ids_batch, pos_ids_batch, vms_batch) in enumerate(batch_loader(batch_size, input_ids, label_ids, mask_ids, pos_ids, vms)):

            vms_batch = torch.LongTensor(vms_batch)

            input_ids_batch = input_ids_batch.to(device)
            label_ids_batch = label_ids_batch.to(device)
            mask_ids_batch = mask_ids_batch.to(device)
            pos_ids_batch = pos_ids_batch.to(device)
            vms_batch = vms_batch.to(device)

            with torch.no_grad():
                loss, logits = model(input_ids_batch, label_ids_batch, mask_ids_batch, pos_ids_batch, vms_batch)
            logits = nn.Softmax(dim=1)(logits)
            if i == 0:
                logits_all=logits
            if i >= 1:
                logits_all=torch.cat((logits_all,logits),0)

        order = -1
        gold = []
        for i in range(len(dataset)):
            qid = dataset[i][-1]
            label = dataset[i][1]
            if qid == order: 
                j += 1
                if label == 1:
                    gold.append((qid,j))
            else:
                order = qid
                j = 0
                if label == 1:
                    gold.append((qid,j))

        label_order = []
        order = -1
        for i in range(len(gold)):
            if gold[i][0] == order:
                templist.append(gold[i][1])
            elif gold[i][0] != order:
                order=gold[i][0]
                if i > 0:
                    label_order.append(templist)
                templist = []
                templist.append(gold[i][1])
        label_order.append(templist)

        order = -1
        score_list = []
        for i in range(len(logits_all)):
            score = float(logits_all[i][1])
            qid=int(dataset[i][-1])
            if qid == order:
                templist.append(score)
            else:
                order = qid
                if i > 0:
                    score_list.append(templist)
                templist = []
                templist.append(score)
        score_list.append(templist)

        rank = []
        pred = []
        print(len(score_list))
        print(len(label_order))
        for i in range(len(score_list)):
            if len(label_order[i])==1:
                if label_order[i][0] < len(score_list[i]):
                    true_score = score_list[i][label_order[i][0]]
                    score_list[i].sort(reverse=True)
                    for j in range(len(score_list[i])):
                        if score_list[i][j] == true_score:
                            rank.append(1 / (j + 1))
                else:
                    rank.append(0)

            else:
                true_rank = len(score_list[i])
                for k in range(len(label_order[i])):
                    if label_order[i][k] < len(score_list[i]):
                        true_score = score_list[i][label_order[i][k]]
                        temp = sorted(score_list[i],reverse=True)
                        for j in range(len(temp)):
                            if temp[j] == true_score:
                                if j < true_rank:
                                    true_rank = j
                if true_rank < len(score_list[i]):
                    rank.append(1 / (true_rank + 1))
                else:
                    rank.append(0)
        MRR = sum(rank) / len(rank)
        print("MRR", MRR)
        return MRR





In [25]:
# Training phase.
print("Start training.")
trainset = read_dataset(args.train_path, workers_num=args.workers_num)
print("Shuffling dataset")
random.shuffle(trainset)
instances_num = len(trainset)
batch_size = args.batch_size

Start training.
Loading sentences from ./huawei_train.csv
There are 5324 sentence in total. We use 1 processes to inject knowledge into sentences.
Progress of process 0: 0/5324
Shuffling dataset


In [26]:
print("Trans data to tensor.")
print("input_ids")
input_ids = torch.LongTensor([example[0] for example in trainset])
print("label_ids")
label_ids = torch.LongTensor([example[1] for example in trainset])
print("mask_ids")
mask_ids = torch.LongTensor([example[2] for example in trainset])
print("pos_ids")
pos_ids = torch.LongTensor([example[3] for example in trainset])
print("vms")
vms = [example[4] for example in trainset]

train_steps = int(instances_num * args.epochs_num / batch_size) + 1

print("Batch size: ", batch_size)
print("The number of training instances:", instances_num)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]
optimizer = BertAdam(optimizer_grouped_parameters, lr=args.learning_rate, warmup=args.warmup, t_total=train_steps)

total_loss = 0.
result = 0.0
best_result = 0.0

Trans data to tensor.
input_ids
label_ids
mask_ids
pos_ids
vms
Batch size:  30
The number of training instances: 5324


# Begin model training

In [27]:
import wandb
wandb.init(config=args)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


W&B Run: https://app.wandb.ai/ed3203/K-BERT/runs/c6rsu475

In [28]:
wandb.watch(model)

In [29]:

for epoch in range(1, args.epochs_num+1):
    model.train()
    for i, (input_ids_batch, label_ids_batch, mask_ids_batch, pos_ids_batch, vms_batch) in enumerate(batch_loader(batch_size, input_ids, label_ids, mask_ids, pos_ids, vms)):
        model.zero_grad()

        vms_batch = torch.LongTensor(vms_batch)

        input_ids_batch = input_ids_batch.to(device)
        label_ids_batch = label_ids_batch.to(device)
        mask_ids_batch = mask_ids_batch.to(device)
        pos_ids_batch = pos_ids_batch.to(device)
        vms_batch = vms_batch.to(device)

        loss, _ = model(input_ids_batch, label_ids_batch, mask_ids_batch, pos=pos_ids_batch, vm=vms_batch)
        if torch.cuda.device_count() > 1:
            loss = torch.mean(loss)
        total_loss += loss.item()
        if (i + 1) % args.report_steps == 0:
            ave_loss = total_loss / args.report_steps
            print("Epoch id: {}, Training steps: {}, Avg loss: {:.3f}".format(epoch, i+1, ave_loss))
            sys.stdout.flush()
            total_loss = 0.
            wandb.log({"train_loss":ave_loss})
        loss.backward()
        optimizer.step()

    print("Start evaluation on dev dataset.")
    result = evaluate(args, False)
    wandb.log({"acc": result})
    if result > best_result:
        best_result = result
        save_model(model, args.output_model_path)
    else:
        continue

    print("Start evaluation on test dataset.")
    evaluate(args, True)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
/opt/conda/conda-bld/pytorch_1587428270644/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Epoch id: 1, Training steps: 100, Avg loss: 5.559
Start evaluation on dev dataset.
Loading sentences from ./huawei_test.csv
There are 2190 sentence in total. We use 1 processes to inject knowledge into sentences.
Progress of process 0: 0/2190
not MMR
Acc. (Correct/Total): 0.0785 (172/2190) 
Start evaluation on test dataset.
Loading sentences from ./huawei_test.csv
There are 2190 sentence in total. We use 1 processes to inject knowledge into sentences.
Progress of process 0: 0/2190
The number of evaluation instances:  2190
not MMR
Acc. (Correct/Total): 0.0785 (172/2190) 
Epoch id: 2, Training steps: 100, Avg loss: 9.062
Start evaluation on dev dataset.
Loading sentences from ./huawei_test.csv
There are 2190 sentence in total. We use 1 processes to inject knowledge into sentences.
Progress of process 0: 0/2190
not MMR
Acc. (Correct/Total): 0.2429 (532/2190) 
Start evaluation on test dataset.
Loading sentences from ./huawei_test.csv
There are 2190 sentence in total. We use 1 processes to 

KeyboardInterrupt: 

In [ ]:
result = evaluate(args, False)

In [ ]:
result

In [ ]:
kg.entities

In [ ]:
len(kg.entities)

In [ ]:
for i in set(kg.entities):
    print(i)

In [ ]:
for i in kg.segment_vocab[:200]:
    print(i)

# Final evaluation

In [ ]:
# Evaluation phase.
print("Final evaluation on the test dataset.")

if torch.cuda.device_count() > 1:
    model.module.load_state_dict(torch.load(args.output_model_path))
else:
    model.load_state_dict(torch.load(args.output_model_path))
evaluate(args, True)


In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()

## TRAIN ANALYSIS

In [ ]:
train = pd.read_csv(args.train_path, sep = '\t')

In [ ]:
train.head()

In [ ]:
for i in kg.entities:
    if np.any([True for e in ['歌手达明一派', '歌手达明', '性质技术用语'] if e in i]):
        print(i)

In [ ]:
train.text_a[:3].values

In [ ]:
kg.lookup_table.get(token, [])

In [ ]:
# kg.entities

In [ ]:
tok = pkuseg.pkuseg(model_name="default", postag=True)#, user_dict=self.segment_vocab)

In [ ]:
print(train.text_a[0])

In [ ]:
#with pos tag
tok = pkuseg.pkuseg(model_name="default", postag=True)#, user_dict=self.segment_vocab)
p= tok.cut('怎么申诉更改华为账号密码呀，我之前的手机停用了，密码也忘了')

In [ ]:
t = ['zenme how','shensu appeal','genggai change','huawei huawei','zhanghao account number','mima password','ya yeah',',','wo I','zhiqian prior to','of','shouji cellphone','ting young deactivate','le off/past tense',',','mima password','ye and also','wang forget','le past tense']
for c, i in enumerate(p):
    print(i, t[c])

In [ ]:
#without pos tag
tok = pkuseg.pkuseg(model_name="default", postag=False)#, user_dict=self.segment_vocab)
tok.cut('怎么申诉更改华为账号密码呀，我之前的手机停用了，密码也忘了')

## ENGLISH DBPEDIA INSPECT

In [ ]:
os.getcwd()

In [ ]:
os.listdir('./brain/kgs/english')

In [ ]:
db = pd.read_csv('./brain/kgs/english/Device.csv')

In [ ]:
db.URI.values.shape

In [ ]:
db.columns.nunique()

In [ ]:
db.columns.unique().tolist()

In [ ]:
[i.split('resource/')[1] for i in db.manufacturer.unique() if type(i)==str and 'resource' in i]

In [ ]:
db.head(n=20)

In [ ]:
for i in db['rdf-schema#label'].values[:]:
    i=str(i).lower()
    if 'huawei' in i:
        print(i)

In [ ]:
for i in db['URI'].values[:]:
    i=str(i).lower()
    if 'huawei' in i:
        print(i)

In [ ]:

drop = ['acceleration','author_label','author','automobileModel','background','blockAlloy_label','blockAlloy','co2Emission','compressionRatio',
       'computingPlatform_label','configuration','']
for i in drop:
    try:
        db = db.loc[db[i].isnull()]
        db.drop(labels=[i], axis=1, inplace=True, errors='ignore')
    except AttributeError as e:
        print(i,e)

In [ ]:
for c in db.columns:
    n = db[c].nunique()
    if n <200:
        print(c, n, db[c].unique())
    else:
        print(c,n)
    print('-'*10)

# cndbpedia

In [ ]:
args.kg_name

In [ ]:
os.listdir('./brain/kgs/')

In [ ]:
sa = []
pa = []
oa = []

with open(spo_path, 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        try:
            subj, pred, obje = line.strip().split("\t")    
            sa.append(subj)
            pa.append(pred)
            oa.append(obje)
        except:
            pass
#         break

In [ ]:
len(set(sa)), len(set(pa)), len(set(oa))

In [ ]:
pd.Series(pa).value_counts().head(n=30).index.tolist()

In [ ]:
pd.Series(oa).value_counts().head(n=30)#.index.tolist()

In [ ]:
spo_path

In [ ]:
f= open(spo_path, 'r', encoding='utf-8')

In [ ]:
banned_pred = ['连载网站', '作者', '书名', '出版社']
banned_obj = ['起点中文网','晋江文学城','晋江文学网','小说阅读网','红袖添香网','云中书城']

In [ ]:
# n= open('./brain/kgs/short_CnDbpedia.spo', 'w', encoding='utf-8')
# spo_path=f'./brain/kgs/{args.kg_name}.spo'
# f= open(spo_path, 'r', encoding='utf-8')
# for line in tqdm(f):
#     try:
#         s, p, o = line.strip().split("\t") 
#         if p not in banned_pred and o not in banned_obj:
#             n.write(f'{s}\t{p}\t{o}\n')
# #         print(s,p,o)
#     except:
#         pass
# #     break

In [ ]:
n.close()

In [ ]:
sa = []
pa = []
oa = []

f= open('./brain/kgs/short_CnDbpedia.spo', 'r', encoding='utf-8')
for line in tqdm(f):
    try:
        subj, pred, obje = line.strip().split("\t")    
        sa.append(subj)
        pa.append(pred)
        oa.append(obje)
    except:
        pass

In [ ]:
print(f.read())

In [ ]:
pd.Series(oa).value_counts().head(n=30)

In [ ]:
pd.Series(pa).value_counts().head(n=30).index.tolist()